## Домашнее задание

1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Набор данных имеет 6 признаков и одну целевую переменную (необходимо определить является компания банкротом).

Attribute Information: (P=Positive,A-Average,N-negative,B-Bankruptcy,NB-Non-Bankruptcy)

1. Industrial Risk: {P,A,N}
2. Management Risk: {P,A,N}
3. Financial Flexibility: {P,A,N}
4. Credibility: {P,A,N}
5. Competitiveness: {P,A,N}
6. Operating Risk: {P,A,N}
7. Class: {B,NB}


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv("./Qualitative_Bankruptcy/Qualitative_Bankruptcy.data.txt", header=None)
data.columns=['Industrial Risk', 'Management Risk', 'Financial Flexibility', 
              'Credibility','Competitiveness', 'Operating Risk','Class']
data.head(3)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,Class
0,P,P,A,A,A,P,NB
1,N,N,A,A,A,N,NB
2,A,A,A,A,A,A,NB


In [3]:
print(data.shape)

(250, 7)


2.*Сделать feature engineering

In [4]:
mapping = {'P': 1,'A': 0,'N': -1,'B': -1,'NB': 1}

df=pd.DataFrame()

for column in data.columns:
    df[column]=data[column].map(mapping)
    
df['F1'] = (df['Industrial Risk']+df['Management Risk']+df['Operating Risk'])
df['F2'] = (df['Credibility']+df['Competitiveness'])

cols = list(df.columns)
a, b = cols.index('Class'), cols.index('F2')
cols[b], cols[a] = cols[a], cols[b]
df = df[cols]

df.head(3)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,F2,F1,Class
0,1,1,0,0,0,1,0,3,1
1,-1,-1,0,0,0,-1,0,-3,1
2,0,0,0,0,0,0,0,0,1


In [5]:
df['Class'].value_counts()

 1    143
-1    107
Name: Class, dtype: int64

1 - функционирующая компания;

-1 - обанкротившаяся компания;

3.Обучить любой классификатор (какой вам нравится)

In [6]:
from sklearn.model_selection import train_test_split

y_data = df['Class']
x_data = df.drop(columns='Class')


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [8]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return [f1,roc,rec,prc]

result= evaluate_results(y_test, y_predict)
result.append('step4')
A = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 100.00%
roc: 100.00%
recall: 100.00%
precision: 100.00%


4. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть

In [9]:
share = .05

mod_data = df.copy()
P = np.where(mod_data['Class'].values == 1)[0]
np.random.shuffle(P)
P_len = int(np.ceil(share * len(P)))
print(f'Using {P_len}/{len(P)} as positives and unlabeling the rest')
P_sample = P[:P_len]

Using 8/143 as positives and unlabeling the rest


In [10]:
mod_data['class_test'] = -1
mod_data.loc[P_sample,'class_test'] = 1
print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    242
 1      8
Name: class_test, dtype: int64


In [ ]:
x_data = mod_data.iloc[:,:-2].values
y_labeled = mod_data.class_test.values
y_positive = mod_data.Class.values

5. Применить random negative sampling для построения классификатора в новых условиях

In [11]:
mod_data = mod_data.sample(frac=1)

neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(8, 10) (8, 10)


In [12]:
model = KNeighborsClassifier(n_neighbors=2)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

result= evaluate_results(sample_test.iloc[:,-2].values, y_predict)
result.append('last step')
B = pd.DataFrame([result],columns=['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 96.03%
roc: 96.18%
recall: 92.37%
precision: 100.00%


6. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик)

In [13]:
AB = A.append(B)
AB = AB[['Model','f1', 'roc', 'recall', 'precision']]
AB

,Model,f1,roc,recall,precision
0,step4,1.000000,1.000000,1.000000,1.0
0,last step,0.960317,0.961832,0.923664,1.0
